In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

num_epochs = 2
num_classes = 10
batch_size = 4
learning_rate = 0.001

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=4, shuffle=True, num_workers=2
)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=4, shuffle=False, num_workers=2
)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
)

Files already downloaded and verified
Files already downloaded and verified


### Defining the Neural Network

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # layers
        self.conv1 = nn.Conv2d(3, 6, 5) # 3 input channels, 6 output channels, 5x5 kernel
        self.pool = nn.MaxPool2d(2, 2) # 2x2 kernel, stride 2
        self.conv2 = nn.Conv2d(6, 16, 5) # 6 input channels, 16 output channels, 5x5 kernel
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 16 * 5 * 5 input features, 120 output features
        self.fc2 = nn.Linear(120, 84) # 120 input features, 84 output features
        self.fc3 = nn.Linear(84, 10) # 84 input features, 10 output features

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # conv1 -> relu -> pool
        x = self.pool(F.relu(self.conv2(x))) # conv2 -> relu -> pool
        x = x.view(-1, 16 * 5 * 5) # flatten
        x = F.relu(self.fc1(x)) # fc1 -> relu
        x = F.relu(self.fc2(x)) # fc2 -> relu
        x = self.fc3(x) # fc3
        return x
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
#if model is on GPU, copy weights to GPU
if device == "cuda:0":
    model.cuda()


In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # cross entropy loss
optimizer = optim.SGD(model.parameters(), lr=learning_rate), # SGD with momentum


In [4]:
from tqdm import trange

def train(model, device, train_loader, criterion, optimizer, epoch):
    # Set the model into training mode
    model.train()
    # instantiate tqdm object with training data
    pbar = trange(int(len(train_loader)), ncols = 70)
    # Loop over each batch from the training set
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        targets = targets.to(device)

        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Calculate loss
        loss = criterion(output, targets)
        
        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()
        

        if batch_idx % 100 == 0:
            pbar.update(100)
            # Print loss (uncomment lines below once implemented)
            pbar.set_description('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            # Save model
            torch.save(model.state_dict(), "model.cptk")
            # Save optimizer
            torch.save(optimizer.state_dict(), "optimizer.cptk")

    pbar.close()

def test(model, device, test_loader, criterion):
    # Set the model into evaluation mode
    model.eval()
    
    # Variables for tracking loss and accuracy
    test_loss = 0
    correct = 0
    
    # Disable gradient computation
    with torch.no_grad():
        for data, targets in test_loader:
            # Copy data to GPU if needed
            data = data.to(device)
            targets = targets.to(device)

            # Forward pass
            output = model(data)

            # Update the total loss
            test_loss += criterion(output, targets).item()

            # Get the index of the max log-probability
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(targets.view_as(pred)).sum().item()

    # Compute the average test loss
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    # Print loss and accuracy
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))


In [7]:

# if there is saved model or optimizer, load it
try:
    model.load_state_dict(torch.load("model.cptk"))
    optimizer.load_state_dict(torch.load("optimizer.cptk")[0])
    print("Loaded model and optimizer")
except FileNotFoundError:
    print("No model or optimizer found")
except Exception as e:
    print("Error occurred while loading model or optimizer:", str(e))


for epoch in range(num_epochs):
    train(model, device, train_loader, criterion, optimizer, epoch)
    test(model, device, test_loader, criterion)


Error occurred while loading model or optimizer: 'tuple' object has no attribute 'load_state_dict'


AttributeError: 'tuple' object has no attribute 'zero_grad'